In [2]:
from IPython.core.display import display
import pandas as pd
import re
import numpy as np
import nltk
import ssl
from sklearn.compose import ColumnTransformer


try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2
import regex
from sklearn.neighbors import KNeighborsClassifier

df = pd.read_csv("../data/interim/ecommerce_data-cleaned-0.2.2.csv", index_col=0, encoding='utf-8')
display(df)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/vaibhavidharashivkar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,brand,name,description,category_raw,price_raw,discount_raw,price_range
0,La Costeï¿½ï¿½a,"La Costena Chipotle Peppers, 7 OZ (Pack of 12)",We aim to show you accurate product informati...,"Food | Meal Solutions, Grains & Pasta | Canned...",31.93,31.93,"(30, 35]"
1,Equate,Equate Triamcinolone Acetonide Nasal Allergy S...,We aim to show you accurate product informati...,Health | Equate | Equate Allergy | Equate Sinu...,10.48,10.48,"(10, 20]"
2,AduroSmart ERIA,AduroSmart ERIA Soft White Smart A19 Light Bul...,We aim to show you accurate product informati...,Electronics | Smart Home | Smart Energy and Li...,10.99,10.99,"(10, 20]"
3,lowrider,"24"" Classic Adjustable Balloon Fender Set Chro...",We aim to show you accurate product informati...,Sports & Outdoors | Bikes | Bike Accessories |...,38.59,38.59,"(35, 40]"
4,Anself,Elephant Shape Silicone Drinkware Portable Sil...,We aim to show you accurate product informati...,Baby | Feeding | Sippy Cups: Alternatives to P...,5.81,5.81,"(0, 10]"
...,...,...,...,...,...,...,...
29994,NineChef,Sheng Xiang Zhen (ShengXiangZhen) Snack + OneN...,We aim to show you accurate product informati...,"Food | Snacks, Cookies & Chips | Chips & Crisp...",45.99,45.99,"(45, 50]"
29996,Shock Sox,Shock Sox Fork Seal Guards 29-36mm Fork Tube 4...,We aim to show you accurate product informati...,Sports & Outdoors | Bikes | Bike Components | ...,33.25,33.25,"(30, 35]"
29997,Princes,Princes Gooseberries 300g,We aim to show you accurate product informati...,"Food | Meal Solutions, Grains & Pasta | Canned...",8.88,8.88,"(0, 10]"
29998,Create Ion,Create Ion Grace 3/4 Inches Straight Hair Iron...,We aim to show you accurate product informati...,Beauty | Hair Care | Hair Styling Tools | Flat...,50.00,24.50,"(45, 50]"


In [3]:
display(df.info())
display(df.brand.str)
display(df.brand.str.encode('utf-8'))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29604 entries, 0 to 29999
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   brand         29045 non-null  object 
 1   name          29604 non-null  object 
 2   description   29552 non-null  object 
 3   category_raw  29588 non-null  object 
 4   price_raw     29604 non-null  float64
 5   discount_raw  29604 non-null  float64
 6   price_range   29604 non-null  object 
dtypes: float64(2), object(5)
memory usage: 1.8+ MB


None

0        b'La Coste\xc3\xaf\xc2\xbf\xc2\xbd\xc3\xaf\xc2...
1                                                b'Equate'
2                                       b'AduroSmart ERIA'
3                                              b'lowrider'
4                                                b'Anself'
                               ...                        
29994                                          b'NineChef'
29996                                         b'Shock Sox'
29997                                           b'Princes'
29998                                        b'Create Ion'
29999                                      b'Takuminowaza'
Name: brand, Length: 29604, dtype: object

In [4]:
def get_range_label(price):
    value = np.round(price, decimals=1)
    if value <= 10:
        return 0
    elif 10 < value <= 20:
        return 1
    elif 20 < value <= 25:
        return 2
    elif 25 < value <= 30:
        return 3
    elif 30 < value <= 35:
        return 4
    elif 35 < value <= 40:
        return 5
    elif 40 < value <= 45:
        return 6
    elif 45 < value <= 50:
        return 7
    elif 50 < value <= 100:
        return 8
    else:
        return 9

df['labels'] = df['price_raw'].apply(lambda x: get_range_label(x))

In [5]:
stemmer = PorterStemmer()
words = stopwords.words("english")
# words = []
# encode_text = lambda x: x.encode() 
cleaned_text = lambda x: " ".join([stemmer.stem(i) for i in regex.sub("[^a-zA-Z0-9]", " ", x).split() if i not in words]).lower()
df['cleaned_name'] = df.name.apply(cleaned_text)
df['cleaned_brand'] = df.brand.fillna("").apply(cleaned_text)
df['cleaned_description'] = df.description.fillna("").apply(cleaned_text)
df['cleaned_category_raw'] = df.category_raw.fillna("").apply(cleaned_text)
display(df)

,brand,name,description,category_raw,price_raw,discount_raw,price_range,labels,cleaned_name,cleaned_brand,cleaned_description,cleaned_category_raw
0,La Costeï¿½ï¿½a,"La Costena Chipotle Peppers, 7 OZ (Pack of 12)",We aim to show you accurate product informati...,"Food | Meal Solutions, Grains & Pasta | Canned...",31.93,31.93,"(30, 35]",4,la costena chipotl pepper 7 oz pack 12,la cost,we aim show accur product inform manufactur su...,food meal solut grain pasta can good can veget
1,Equate,Equate Triamcinolone Acetonide Nasal Allergy S...,We aim to show you accurate product informati...,Health | Equate | Equate Allergy | Equate Sinu...,10.48,10.48,"(10, 20]",1,equat triamcinolon acetonid nasal allergi spra...,equat,we aim show accur product inform manufactur su...,health equat equat allergi equat sinu congest ...
2,AduroSmart ERIA,AduroSmart ERIA Soft White Smart A19 Light Bul...,We aim to show you accurate product informati...,Electronics | Smart Home | Smart Energy and Li...,10.99,10.99,"(10, 20]",1,adurosmart eria soft white smart a19 light bul...,adurosmart eria,we aim show accur product inform manufactur su...,electron smart home smart energi light smart l...
3,lowrider,"24"" Classic Adjustable Balloon Fender Set Chro...",We aim to show you accurate product informati...,Sports & Outdoors | Bikes | Bike Accessories |...,38.59,38.59,"(35, 40]",5,24 classic adjust balloon fender set chrome bi...,lowrid,we aim show accur product inform manufactur su...,sport outdoor bike bike accessori bike fender
4,Anself,Elephant Shape Silicone Drinkware Portable Sil...,We aim to show you accurate product informati...,Baby | Feeding | Sippy Cups: Alternatives to P...,5.81,5.81,"(0, 10]",0,eleph shape silicon drinkwar portabl silicon c...,anself,we aim show accur product inform manufactur su...,babi feed sippi cup altern plastic
...,...,...,...,...,...,...,...,...,...,...,...,...
29994,NineChef,Sheng Xiang Zhen (ShengXiangZhen) Snack + OneN...,We aim to show you accurate product informati...,"Food | Snacks, Cookies & Chips | Chips & Crisp...",45.99,45.99,"(45, 50]",7,sheng xiang zhen shengxiangzhen snack onenin c...,ninechef,we aim show accur product inform manufactur su...,food snack cooki chip chip crisp chip crisp
29996,Shock Sox,Shock Sox Fork Seal Guards 29-36mm Fork Tube 4...,We aim to show you accurate product informati...,Sports & Outdoors | Bikes | Bike Components | ...,33.25,33.25,"(30, 35]",4,shock sox fork seal guard 29 36mm fork tube 4 ...,shock sox,we aim show accur product inform manufactur su...,sport outdoor bike bike compon bike fork
29997,Princes,Princes Gooseberries 300g,We aim to show you accurate product informati...,"Food | Meal Solutions, Grains & Pasta | Canned...",8.88,8.88,"(0, 10]",0,princ gooseberri 300g,princ,we aim show accur product inform manufactur su...,food meal solut grain pasta can good can fruit
29998,Create Ion,Create Ion Grace 3/4 Inches Straight Hair Iron...,We aim to show you accurate product informati...,Beauty | Hair Care | Hair Styling Tools | Flat...,50.00,24.50,"(45, 50]",7,creat ion grace 3 4 inch straight hair iron ci...,creat ion,we aim show accur product inform manufactur su...,beauti hair care hair style tool flat iron hai...


In [6]:
vectorizer = TfidfVectorizer(min_df= 3, stop_words="english", sublinear_tf=True, norm='l2', ngram_range=(1, 2))
final_features_name = vectorizer.fit_transform(df['cleaned_name']).toarray()
final_features_brand = vectorizer.fit_transform(df['cleaned_brand']).toarray()
final_features_description = vectorizer.fit_transform(df['cleaned_description']).toarray()
final_features_category = vectorizer.fit_transform(df['cleaned_category_raw']).toarray()
# final_features.shape

In [7]:
X = [df['cleaned_name'], df['cleaned_description'], df['cleaned_brand'], df['cleaned_category_raw']]

for i in X:
    X_train_KNN, X_test_KNN, y_train_KNN, y_test_KNN = train_test_split(i, df['labels'], test_size=0.25)

    pipeline = Pipeline([('vect', vectorizer),
                     ('chi',  SelectKBest(chi2, k=1200)),
                     ('clf', KNeighborsClassifier())])

    model = pipeline.fit(X_train_KNN, y_train_KNN)

    ytest_KNN = np.array(y_test_KNN)

    print(classification_report(ytest_KNN, model.predict(X_test_KNN)))
    print(confusion_matrix(ytest_KNN, model.predict(X_test_KNN)))
    

              precision    recall  f1-score   support

           0       0.39      0.48      0.43      1629
           1       0.36      0.40      0.37      1915
           2       0.18      0.07      0.10       604
           3       0.23      0.10      0.13       526
           4       0.05      0.21      0.08       330
           5       0.21      0.07      0.10       319
           6       0.35      0.11      0.16       219
           7       0.15      0.06      0.08       198
           8       0.35      0.27      0.30       832
           9       0.57      0.41      0.47       829

    accuracy                           0.31      7401
   macro avg       0.28      0.22      0.23      7401
weighted avg       0.34      0.31      0.31      7401

[[787 412  34  25 280  11   7   4  41  28]
 [556 758  57  47 370  20   5   4  54  44]
 [153 210  44  19 107  11   1   6  36  17]
 [ 99 171  25  50 105   7   6   4  35  24]
 [ 74  98  14  14  69   3   1   6  34  17]
 [ 60  85  18  14  68  22 

In [8]:
# from sklearn.base import BaseEstimator, TransformerMixin
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.pipeline import Pipeline, FeatureUnion
# 
# class ItemSelector(BaseEstimator, TransformerMixin):
# 
#     def __init__(self, column):
#         self.column = column
# 
#     def fit(self, X, y=None):
#         return self
# 
#     def transform(self, X, y=None):
#         return X[self.column]
# 
# pipeline = Pipeline([
#     ("union", FeatureUnion(
#         transformer_list=[
#             ("cleaned_name", Pipeline([
#                 ("selector", ItemSelector(column="cleaned_name")),
#                 ("tfidf", TfidfVectorizer())
#             ]))
#         ]
#     )),
#     ("classifier", KNeighborsClassifier())
# ])
# 
# X_train_KNN, X_test_KNN, y_train_KNN, y_test_KNN = train_test_split(X, Y, test_size=0.25)
# 
# model = pipeline.fit(training_data, training_data["Category"].tolist())
# 
# predictions = model.predict(test_data)
# probabilities = model.predict_proba(test_data)
# score = model.score(test_data, test_data["Category"].tolist())
# labels = sorted(set(training_labels + test_labels))
# matrix = metrics.confusion_matrix(test_data["Category"].tolist(), predictions, labels=labels)

In [16]:
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier

X = df
X_train_KNN, X_test_KNN, y_train_KNN, y_test_KNN = train_test_split(X, df['labels'], test_size=0.25)
column_trans = ColumnTransformer([('name', vectorizer, 'cleaned_name'),('description', vectorizer, 'cleaned_description'), 
                                  ('brand', vectorizer, 'cleaned_brand'), ('category_raw', vectorizer, 'cleaned_category_raw')],
                                remainder='drop', verbose_feature_names_out=False)

pipeline = Pipeline([('vect', column_trans),
                     ('chi',  SelectKBest(chi2, k=7000)),
                     ('clf', KNeighborsClassifier())])

model = pipeline.fit(X_train_KNN, y_train_KNN)

ytest_KNN = np.array(y_test_KNN)

print(classification_report(ytest_KNN, model.predict(X_test_KNN)))
print(confusion_matrix(ytest_KNN, model.predict(X_test_KNN)))

              precision    recall  f1-score   support

           0       0.55      0.62      0.58      1620
           1       0.39      0.66      0.49      1891
           2       0.32      0.11      0.16       617
           3       0.34      0.11      0.16       491
           4       0.37      0.13      0.19       335
           5       0.38      0.10      0.16       311
           6       0.43      0.12      0.19       209
           7       0.41      0.09      0.15       219
           8       0.43      0.39      0.41       881
           9       0.61      0.68      0.64       827

    accuracy                           0.46      7401
   macro avg       0.42      0.30      0.31      7401
weighted avg       0.44      0.46      0.42      7401

[[1007  549   14    4    4    2    1    2   25   12]
 [ 450 1239   49   25   18    9    1    0   59   41]
 [  99  349   65   26    6    6    2    2   36   26]
 [  59  261   26   53   13    8    2    4   42   23]
 [  39  147   14   14   43   